In [1]:
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X, Z
from qat.core import Observable, Term
from qat.qpus import get_default_qpu, PyLinalg
from qat.plugins import ScipyMinimizePlugin
from math import sqrt, pi

from pysat.formula import CNF

# Resolvendo o problema 3-SAT usando Grover

<!-- Descrever Grover -->
O algoritmo de Grover é um algoritmo quântico que pode ser usado para *busca não estruturada*. O Algoritmo possui a complexidade ótima de $O(\sqrt{N})$ onde $N$ é o tamanho total do espaço da busca.

<!-- Grover não é busca em um banco de dados -->
Quando usa-se Grover para resolver um problema, como quebra de uma chave criptográfica, é comum confundir o seu comportamento. O algoritmo não faz consultas a um banco de dados magicamente codificado, o qual precisaria estar armazenado integralmente no circuito quântico. Em vez disso, dada uma função booleana arbitrária $f:\big\{0,1\big\}^n\rightarrow \{0,1\}$ que seja eficientemente computável, cuja expressão matemática é

$$\begin{equation*}
    f(x) = \left\{
  \begin{array}{ll}
    1, & \hbox{se $x$ resolve o problema,} \\
    0, & \hbox{caso contrário.}
  \end{array}
\right.
\end{equation*}$$
e você quer descobrir o valor $x$ para o qual $f(x) = 1$. Como você consegue computar $f(x)$ você pode rodar $f$ em um computador quântico e usar Grover para encontrar $x$.

Sua relevância como algoritmo é pelo fato de que uma grande parte de problemas interessantes, diga-se a classe NP, possuem soluções que são difíceis de computar mas são fáceis de verificar. Portanto, Grover oferece um ganho quadrático para qualquer método de força-bruta para resolver essa classe de problemas.

## SAT

<!-- Descrever 3-SAT -->
O Problema de Satisfatibilidade Booleana é o problema de determinar se existe uma interpretação que satisfaça uma fórmula Booleana. Ou seja, o problema busca uma sequência de valores VERDADEIRO ou FALSO que possa substituir as variáveis da fórmula Booleana de forma que seu valor final seja VERDADEIRO. Caso exista essa sequência, a fórmula é *satisfazível*. Por outro lado, se não existe essa sequência, a expressão definida pela fórmula é FALSO para todos os valores e é *insatisfazível*.


<!-- Descrever CNF -->
## Forma Normal Conjuntiva

Uma fórmula booleana é formada por variáveis booleanas, também chamadas de literais. A Forma Normal Conjuntiva é uma forma padrão que pode ser usada para representar qualquer sentença lógica. Nessa forma, a sentença lógica é representada por uma conjunção de cláusulas. Uma conjunção ($\wedge$) é representada pela operação lógica E. Uma cláusula é uma disjunção ($\vee$, operação lógica OU) de literais. Por exemplo, a expressão

$$ (A \vee B \vee C) \wedge (D \vee E) \wedge (F \vee G)$$

Se traduz por "A ou B ou C e D ou E e F ou G". A FNC é fácil de se trabalhar e manipular e é possível transformar qualquer sentença lógica em uma forma normal conjuntiva aplicando-se regras de inferência e algumas transformações.

Em sua formulação 3-SAT, todas as cláusulas possuem 3 literais.

<!-- Usando Grover em 3-SAT -->
Grover aplicado ao 3-SAT possui complexidade $O(2^{\frac{n}{2}}) = O(1.414^n)$.
Entretanto, para o problema do 3-SAT, o melhor algoritmo clássico conhecido possui a complexidade de $O(1.307^n)$.
Isso é porque problemas NP-completos possuem uma estrutura que pode ser explorada para se obter soluções, enquanto o algoritmo de Grover é ótimo para problemas sem estrutura.
Apesar de não parecer vantajoso usar Grover para resolver 3-SAT, as técnicas usadas aqui podem ser expandidas para casos gerais, como o k-SAT, no qual Grover supera o melhor algoritmo clássico atual.

In [3]:
# remove one of the clauses
cnf = CNF(from_file='example.cnf')
#cnf = CNF(from_file='example2.cnf')
#cnf = CNF(from_file='uf20-01.cnf')
num_literals = cnf.nv

print(cnf.clauses)
print(num_literals)

FileNotFoundError: [Errno 2] No such file or directory: 'example.cnf'

In [ ]:
def oracle(num_qubits, cnf):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(len(cnf.clauses)+1)
    routine.set_ancillae(ancilla)

    for i, clause in enumerate(cnf.clauses):
        qubits = [abs(q) for q in clause]
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])
        X.ctrl(len(qubits))([wires[n-1] for n in qubits], ancilla[i+1])
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])

    for qubit in ancilla[1:]:
        X(qubit)
    X.ctrl(len(ancilla)-1)(ancilla[1:], ancilla[0])
    for qubit in ancilla[1:]:
        X(qubit)

    for i, clause in enumerate(reversed(cnf.clauses)):
        qubits = [abs(q) for q in clause]
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])
        X.ctrl(len(qubits))([wires[n-1] for n in qubits], ancilla[-i+len(cnf.clauses)])
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])


    return routine

In [ ]:
def diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in wires:
        H(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in wires:
        X(qubit)
    # Do multi-controlled-Z gate
    X.ctrl(len(wires))(wires, ancilla)
    # Apply transformation |11..1> -> |00..0>
    for qubit in wires:
        X(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in wires:
        H(qubit)

    return routine

In [ ]:
def check_result(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(len(cnf.clauses)+1)
    routine.set_ancillae(ancilla)

    for i, clause in enumerate(cnf.clauses):
        qubits = [abs(q) for q in clause]
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])
        X.ctrl(len(qubits))([wires[n-1] for n in qubits], ancilla[i+1])
        for control in clause:
            if control < 0:
                X(wires[abs(control)-1])

    for qubit in ancilla[1:]:
        X(qubit)
    X.ctrl(len(ancilla)-1)(ancilla[1:], ancilla[0])
    for qubit in ancilla[1:]:
        X(qubit)

    return routine

In [ ]:
def initial_state(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    for q in wires:
        H(q)
    X(ancilla)
    H(ancilla)

    return routine

def clean_ancilla(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)

    H(ancilla)
    X(ancilla)

    return routine

In [ ]:
qprog = Program()
num_qubits = num_literals
# --------------------------
# Initial state preparation
# --------------------------
qbits = qprog.qalloc(num_qubits)
cbits = qprog.calloc(num_qubits)

initial = initial_state(num_qubits)
oracle_operator = oracle(num_qubits, cnf)
diffusion_operator = diffusion(num_qubits)
clean = clean_ancilla(num_qubits)
check = check_result(num_qubits)

# -------------------------
# Applying Grover Algorithm
# -------------------------
initial(qbits)
for step in range(int((pi/4)*sqrt(2**num_qubits))):
    oracle_operator(qbits)
    diffusion_operator(qbits)
clean(qbits)
check(qbits)

### Visualização do Circuito

In [ ]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Create a job
job = circuit.to_job(nbshots=1000)

qpu = PyLinalg()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
     print("State %s: probability %s +/- %s amplitude %s " % (sample.state, sample.probability, sample.err, sample.amplitude))